# Meshing

<!-- SUMMARY: Description of various meshings. Focus on the Turbo Meshing (based on an internal grid, possibly rotated) -->

<!-- CATEGORY: Meshing_Technique -->

In [ ]:
import numpy as np
import os
import pandas as pd
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc

gdoc.setNoScroll()

## Standard Meshing based on an Irregular Data Set

We construct a Meshing Standard based on a set of Data Points 

In [ ]:
nech = 40
extendmin = [0,0]
extendmax = [150,100]
data = gl.Db.createFromBox(nech,extendmin, extendmax)
data

In [ ]:
gp.setDefaultGeographic(dims=[7,7])

ax = data.plot()
ax.decoration(title="Display of Data Set")

Creating the Meshing

## Turbo Meshing

Instead we can use Turbo Meshing to cover an area, without specifically honoring each datum

In [ ]:
mesh3 = gl.MeshETurbo()
err = mesh3.initFromExtend(extendmin, extendmax, [5,5])
mesh3.display()

In [ ]:
ax = mesh3.plot()
ax.decoration(title="Turbo Meshing on the whole area")
ax = data.plot(color="black")

We can create a regular grid covering the same area (based on the data set)

In [ ]:
grid = gl.DbGrid()
err = grid.resetCoveringDb(data, [50,50])

Define a Polygon as the Convex hull of the data

In [ ]:
polygon = gl.Polygons()
err = polygon.resetFromDb(data)

Use the Polygon to mask off some nodes of the regular grid, located too far from the data

In [ ]:
err = gl.db_polygon(grid, polygon)

We create a Turbo Meshing from the grid (which contains a selection)

In [ ]:
mesh4 = gl.MeshETurbo(grid)
mesh4.display()

In [ ]:
ax = mesh4.plot()
ax = data.plot(color="black")
ax = polygon.plot()
ax.decoration(title="Turbo Meshing restricted to the Convex Hull of Data")

## Read and Write in Neutral File

Testing the read and write into a Neutral File (with masked meshes)

In [ ]:
gl.ASerializable.setContainerName(True)
gl.ASerializable.setPrefixName("Tuto-Meshing")

err = mesh4.dumpToNF("Mesh_masked")
mesh5 = gl.MeshETurbo.createFromNF("Mesh_masked")
mesh5.display()

In [ ]:
mesh5.display()

## Turbo Meshing on Rotated Grid

In [ ]:
grid = gl.DbGrid.create(nx=[6,4], dx=[1.,5.], x0=[10.,20.], angles=[-80.,0.])
grid.display()
ax = gp.point(grid,color="black")

In [ ]:
model = gl.Model.createFromParam(gl.ECov.CUBIC,ranges=[10.,5.], angles=[30.,0.])
model.display()

In [ ]:
mesh6 = gl.MeshETurbo()
mesh6.initFromCova(model.getCova(0),grid,ratio=10,nbExt=2,flagNoStatRot=False,useSel=True)
mesh6.display()

In [ ]:
ax = mesh6.plot()
ax = gp.point(grid,color="black")
ax.decoration(title="Turbo Meshing for Rotated Grid")